In [ ]:
import cv2
import matplotlib.pyplot as plt
import inference as inf
import pandas as pd
import torch
from pathlib import Path
from configs.config import load_config
from preprocessing.dataset_split import _resolve_allowlist
from models import model as model_mod


Config

In [ ]:
config = load_config("configs/default.yaml")


Paths

In [ ]:
image_name = "P333952_7_1536_256.jpg"
base_name = Path(image_name).stem
image_path = config.paths.images_root / image_name
cropped_annotations_path = config.paths.csv_annot
best_checkpoint_path = Path("runs/20250913-182536/best_map50.pth")

Device

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else: device = torch.device("cpu")

Annotations df

In [ ]:
df = pd.read_csv(cropped_annotations_path)
image_gt = df[df.image_path == image_path.name].copy()

Class names

In [ ]:
allow = _resolve_allowlist(config=config)
class_names = [str(x) for x in allow]
if config.dataset.include_other_class:
    class_names.append(config.dataset.other_class_name)
num_classes = len(class_names)

Create model

In [ ]:
model = model_mod.build_model(
    num_classes=num_classes,
    anchor_sizes=config.model.anchor_sizes,
    aspect_ratios=config.model.aspect_ratios,
    score_thresh=config.model.score_thresh,
    nms_thresh=config.model.nms_thresh,
    detections_per_img=config.model.detections_per_img,
    imagenet_weights=config.model.use_imagenet_weights
)

In [ ]:
model = inf.load_model_from_ckpt(path=best_checkpoint_path, model=model, device=device)

Infer

In [ ]:
metrics, preds_eval, out_path, im_bgr = inf.eval_one_image(
    model=model,
    image_path=str(image_path),
    gt_df_for_image=image_gt,
    class_names=class_names,
    images_root=config.paths.images_root,
    device=device,
    score_thresh=model.score_thresh,
    iou_thr=0.5,
    out_path=f"./outputs/{base_name}_overlay.png"
)

print(metrics)

Visuals

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()